In [ ]:
# --- Capítulo 9: Comparação com Aprendizado Supervisionado (SL-Regressor) ---

print("\n--- Etapa 9.1: Gerando dataset SINTÉTICO (Preços Discretos) para SL ---")

# Verifica se as métricas do simulador (da Etapa 3) existem
if 'region_metrics' not in locals() or 'elasticity_factors' not in locals():
    raise ValueError("ERRO: 'region_metrics' ou 'elasticity_factors' não definidos. Execute a Etapa 3 primeiro.")

# Lista de preços DISCRETOS que este modelo SL testará
price_points = [29.99, 49.99, 79.99, 99.99, 149.99, 199.99]
generated_data_sl = [] # Nova lista para dados SL

# Itera sobre o DataFrame ORIGINAL 'df' (da Etapa 2)
for _, row in df.iterrows():
    region = row['Region']
    metrics = region_metrics.get(region)
    if not metrics: continue

    elasticity = elasticity_factors.get(region, 0.015)

    # Para cada linha do df, simula o lucro para cada um dos 6 preços fixos
    for price in price_points:
        # Lógica de simulação (simplificada do seu notebook SL)
        # Nota: A fórmula aqui é ligeiramente diferente da Etapa 3 do RL (ex: 'price - 50' vs 'price - price_reference')
        # Idealmente, as lógicas do simulador deveriam ser idênticas.
        # Vamos manter a lógica original do seu notebook SL:

        simulated_conversion_rate = metrics['avg_conversion_rate'] * np.exp(-elasticity * (price - 50)) # Ref '50' do notebook SL
        simulated_conversion_rate = max(0, simulated_conversion_rate)
        simulated_conversions = row['Clicks'] * simulated_conversion_rate
        simulated_revenue = simulated_conversions * price
        simulated_marketing_cost = simulated_conversions * metrics['avg_cpa']
        simulated_profit = simulated_revenue - simulated_marketing_cost

        # Adiciona o resultado (Estado + Ação + Recompensa)
        generated_data_sl.append({
            'Region': region,
            'Content_Type': row['Content_Type'],
            'Target_Age': row['Target_Age'],
            'Target_Gender': row['Target_Gender'],
            'Platform': row['Platform'],
            'Budget': row['Budget_Category'],
            'Price_Action': price, # O preço é uma FEATURE
            'Simulated_Profit_Reward': simulated_profit # O lucro é o ALVO
        })

# Cria o DataFrame de treino do SL
df_sl_training = pd.DataFrame(generated_data_sl)
df_sl_training.dropna(inplace=True)
print(f"Dataset para Supervised Learning gerado com {len(df_sl_training)} linhas.")


--- Etapa 9.1: Gerando dataset SINTÉTICO (Preços Discretos) para SL ---


ValueError: ERRO: 'region_metrics' ou 'elasticity_factors' não definidos. Execute a Etapa 3 primeiro.

In [ ]:
# --- Etapa 10: Formatando dados para o modelo de SL (Regressão) ---
print("\n--- Etapa 10: Formatando dados para o modelo de SL ---")

# Importações necessárias para SL
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define o alvo (y) = Lucro
y_sl = df_sl_training['Simulated_Profit_Reward']
# Define as features (X) = Estado + Ação (Preço)
X_sl_temp = df_sl_training.drop('Simulated_Profit_Reward', axis=1)

# One-Hot Encoding para as features de estado categóricas
# 'Price_Action' é mantida como numérica, pois é uma feature
X_sl = pd.get_dummies(X_sl_temp, columns=['Region', 'Content_Type', 'Target_Age', 'Target_Gender', 'Platform', 'Budget'])

# Divide os dados SL em treino e teste
X_train_sl, X_test_sl, y_train_sl, y_test_sl = train_test_split(X_sl, y_sl, test_size=0.2, random_state=42)

print(f"Dados SL divididos em {len(X_train_sl)} para treino e {len(X_test_sl)} para teste.")
print("Features SL de exemplo:\n", X_train_sl.head(1))

In [ ]:
# --- Etapa 11: Treinando o modelo LightGBM para PREVER O LUCRO ---
print("\n--- Etapa 11: Treinando o modelo LightGBM para prever o lucro ---")

# Instancia o REGRESSOR LightGBM
lgbm_regressor = lgb.LGBMRegressor(random_state=42)

# Treina o modelo para aprender a mapear (Estado, Preço) -> Lucro
lgbm_regressor.fit(X_train_sl, y_train_sl)

print("Modelo SL (Regressor) treinado com sucesso!")

In [ ]:
# --- Etapa 12: Avaliando a PRECISÃO do modelo SL no conjunto de teste ---
print("\n--- Etapa 12: Avaliando a precisão do modelo SL no conjunto de teste ---")

# Faz previsões de lucro no conjunto de teste SL
y_pred_sl = lgbm_regressor.predict(X_test_sl)

# Calcula métricas de precisão da regressão
mae = mean_absolute_error(y_test_sl, y_pred_sl)
mse = mean_squared_error(y_test_sl, y_pred_sl)
r2 = r2_score(y_test_sl, y_pred_sl)

print(f"----------- MÉTRICAS DE PRECISÃO (Regressão do Lucro) -----------\n")
print(f"Erro Médio Absoluto (MAE): ${mae:,.2f}")
print(f"Erro Quadrático Médio (MSE): {mse:,.2f}")
print(f"R-quadrado (R²): {r2:.2%}")
print("----------------------------------------------------------------")
print("\n- MAE: Em média, as previsões de lucro do modelo SL erram por este valor.")
print("- R²: O modelo SL consegue explicar esta porcentagem da variação no lucro simulado.")

In [ ]:
# --- Etapa 13: Avaliação da ESTRATÉGIA de precificação SL (Comparação) ---
print("\n--- Etapa 13: Avaliando o desempenho financeiro da estratégia SL ---")

# Função de otimização: testa todos os 6 preços e escolhe o melhor
def get_optimal_price_sl(scenario, model, prices_to_test, original_cols):
    scenario_df = pd.DataFrame([scenario])
    best_price = None
    max_profit = -np.inf

    # Para cada preço possível...
    for price in prices_to_test:
        scenario_df['Price_Action'] = price
        # Prepara o cenário para o modelo
        scenario_onehot = pd.get_dummies(scenario_df)
        scenario_aligned = scenario_onehot.reindex(columns=original_cols, fill_value=0)

        # Usa o modelo treinado para PREVER o lucro
        predicted_profit = model.predict(scenario_aligned)[0]

        # Encontra o preço que dá o maior lucro previsto
        if predicted_profit > max_profit:
            max_profit = predicted_profit
            best_price = price

    return best_price, max_profit

# --- Avaliação Financeira da Política SL ---

# 1. Pega os dados de TESTE *antes* do one-hot encoding
test_data_original_sl = df_sl_training.loc[X_test_sl.index]

# 2. Define as colunas do cenário
scenario_cols = ['Region', 'Content_Type', 'Target_Age', 'Target_Gender', 'Platform', 'Budget']

# 3. Pega os cenários únicos do conjunto de teste
unique_scenarios_sl = test_data_original_sl[scenario_cols].drop_duplicates().to_dict('records')
print(f"Avaliando a estratégia SL em {len(unique_scenarios_sl)} cenários únicos do conjunto de teste...")

# 4. Cria um "mapa da verdade" com o lucro real (simulado) para cada (cenário, preço)
profit_map_sl = test_data_original_sl.set_index(scenario_cols + ['Price_Action'])['Simulated_Profit_Reward'].to_dict()

total_profit_sl = 0
n_scenarios_sl = len(unique_scenarios_sl)

# 5. Itera sobre cada cenário único
for scenario in unique_scenarios_sl:
    # Pergunta ao modelo SL qual é o melhor preço (dos 6 disponíveis)
    best_price_sl, _ = get_optimal_price_sl(scenario, lgbm_regressor, price_points, X_train_sl.columns)

    # Cria a chave para buscar o lucro real (simulado)
    scenario_key_tuple = tuple(scenario[col] for col in scenario_cols)
    full_key = scenario_key_tuple + (best_price_sl,)

    # Busca o lucro real (simulado) para o preço que o modelo escolheu
    realized_profit = profit_map_sl.get(full_key)

    if realized_profit is not None:
        total_profit_sl += realized_profit

# Calcula o lucro médio por decisão
average_profit_sl = total_profit_sl / n_scenarios_sl if n_scenarios_sl > 0 else 0

print(f"\n----------- RESULTADO DA AVALIAÇÃO DA ESTRATÉGIA SL -----------")
print(f"Lucro médio por decisão seguindo a política do SL: ${average_profit_sl:,.2f}")
print("---------------------------------------------------------------")

In [ ]:
# --- Etapa 5: Chamar o LLM para Gerar o Conteúdo (v23 - Teste Simplificado) ---
import google.generativeai as genai # Garante que está importado aqui também
from google.generativeai.types import HarmCategory, HarmBlockThreshold # Para configurações de segurança

# Verifica se a chave foi carregada com sucesso ANTES de tentar usar
if GOOGLE_API_KEY:
    try:
        # Usa um modelo recente e disponível
        model_llm = genai.GenerativeModel('gemini-1.5-flash-latest')
        print("\\n--- Chamando o LLM para gerar o conteúdo (chamada simplificada)... ---")

        # --- TENTATIVA DE CORREÇÃO ---
        # 1. Remover o generation_config explícito por agora
        # 2. Adicionar safety_settings básicos para permitir mais conteúdo (APENAS PARA TESTE)
        safety_settings_for_test = {
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }

        # Faz a chamada à API simplificada
        response = model_llm.generate_content(
            prompt_para_llm,
            safety_settings=safety_settings_for_test # Adiciona as configurações de segurança
            # generation_config removido temporariamente
            )
        # --- FIM DA TENTATIVA DE CORREÇÃO ---
        # Tenta aceder ao texto da resposta
        if response and hasattr(response, 'text'):
             generated_text = response.text
             print("\n--- Conteúdo Gerado pelo LLM ---")
             print(generated_text)
        elif response and response.parts:
             # Às vezes a resposta pode vir em 'parts' se houver bloqueios de segurança
             generated_text = "".join(part.text for part in response.parts)
             print("\n--- Conteúdo Gerado pelo LLM (a partir de 'parts') ---")
             print(generated_text)
             if response.prompt_feedback.block_reason:
                  print(f"\\nAtenção: A resposta pode ter sido bloqueada ou filtrada. Razão: {response.prompt_feedback.block_reason}")
        else:
             print("\\nErro: A resposta da API não continha texto.")
             print("Resposta completa:", response)

    except Exception as e:
      print(f"\n!!! Erro ao chamar a API do LLM: {e} !!!")
      print("   -> Verifique se a API Key é válida.")
      print("   -> Verifique se o nome do modelo ('gemini-1.5-flash-latest') está correto e acessível.")
      print("   -> Verifique a sua conexão à internet.")
      traceback.print_exc()
else:
    # Esta mensagem aparece se o try/except na Etapa 2 falhar
    print("\n!!! A API Key não foi carregada corretamente através dos Secrets. Pulando a chamada ao LLM. !!!")
    print("   -> Verifique as mensagens de erro na Etapa 2 e as instruções para adicionar o Secret.")